In [145]:
#Import libraries
import os
import ee
import geemap
#initialisation of earth engine
ee.Initialize()

In [179]:
#import area of interest
Map=geemap.Map()
aoi_shp = r'.....\uttarkashi.shp'
aoi = geemap.shp_to_ee(aoi_shp)
Map.addLayer(aoi, {}, 'boundary')
#Map.addLayer({'eeObject': aoi_shp, 'color': 'FF0000'}, {}, 'Shapefile Boundary')

Map.centerObject(aoi,9)
Map

Map(center=[30.971881310135245, 78.60297713823928], controls=(WidgetControl(options=['position', 'transparent_…

In [191]:
image= ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
    .filterDate('2013-11-10', '2013-12-31')\
    .filterBounds(aoi)\
    .sort('CLOUD_COVER')\
    #.first()
#print(ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo())
bands=['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
composite = image.median().select(bands)\
    .clip(aoi)
vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

#Map.centerObject(aoi, 9)
Map.addLayer(composite, vis_params, "Landsat-8_2013")
props = geemap.image_props(composite)
props.getInfo()
# Make the training dataset.

points = composite.sample(
    **{
        'region': aoi,
        'scale': 30,
        'numPixels': 20000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)
# Instantiate the clusterer and train it.
n_clusters = 8
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(points)
# Cluster the input using the trained clusterer.
lulc2013 = composite.cluster(clusterer)

# # Display the clusters with random colors.
#Map.addLayer(lulc2013.randomVisualizer(), {}, 'clusters')
legend_keys = ['Forest', 'non_forest']
legend_colors = ['green', 'yellow',]

# Reclassify the map
#result = result.remap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 3, 3, 3, 3])
lulc_2013 = lulc2013.remap([0, 1, 2, 3, 4, 5, 6, 7 ], [1, 2, 2, 1, 1, 2, 1, 2])

Map.addLayer(lulc_2013, {'min': 1, 'max': 2, 'palette': legend_colors}, 'LULC_2013')
forest_2013 = lulc_2013.updateMask(lulc_2013.neq(2))
Map.addLayer(forest_2013, {'palette': 'blue'}, 'forest_2013')
Map

Map(bottom=107634.0, center=[30.970544333076077, 78.62228393554689], controls=(WidgetControl(options=['positio…

In [ ]:
# Export the map as an image
export_params = {
    'image': Map.toImage(),
    'description': 'layout_export',
    'scale': 30,  # Adjust the scale based on your raster resolution
    'region': aoi
}

# Start the export task
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

In [ ]:
geom=aoi.geometry()
out_dir = r'.....'
out_file = os.path.join(out_dir, 'forest_2013.tif')
geemap.ee_export_image(Map.toImage(), filename=out_file, scale=90, region=geom)
#out_file = os.path.join(out_dir, 'landcover_2023.tif')
#geemap.ee_export_image(forest_2023, filename=out_file, scale=90, region=geom)

In [ ]:
#calculate area of region and forest area and print it on the screen
stats = forest_2013.multiply(ee.Image.pixelArea()).reduceRegion(
  reducer= ee.Reducer.sum(),
  geometry= aoi,
  scale= 30,
  maxPixels= 1e13,
  tileScale= 30);

forest_area_2013 = ee.Number(stats.get('sum')).divide(1000000).round();
print('Forest Area (sq.km.) in  2013', forest_area_2013);

In [193]:
sentinel2= ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
    .filterDate('2023-12-01', '2023-12-31')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterBounds(aoi);
    #.first()
    
bands=['B2', 'B3', 'B4', 'B8', 'B11', 'B12']    
composite = sentinel2.median().select(['B8', 'B4', 'B3'])\
    .clip(aoi)
vis_params = {'min': 0, 'max': 3000, 'bands': ['B8', 'B4', 'B3']}

#Map.centerObject(aoi, 9)
Map.addLayer(composite, vis_params, "Sentinel-2_2023")



props = geemap.image_props(composite)
props.getInfo()
# Make the training dataset.
points = composite.sample(
    **{
        'region': aoi,
        'scale': 10,
        'numPixels': 20000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)
# Instantiate the clusterer and train it.
n_clusters = 8
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(points)
# Cluster the input using the trained clusterer.
lulc2023 = composite.cluster(clusterer)

# # Display the clusters with random colors.
#Map.addLayer(lulc2023.randomVisualizer(), {}, 'clusters')
legend_keys = ['Forest', 'Non_forest']
legend_colors = ['Green', 'yellow']

# Reclassify the map
#result = result.remap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [3, 1, 2, 3, 1, 2, 1, 4, 3, 3, 5, 11, 12, 13, 14, 15])
lulc_2023 = lulc2023.remap([0, 1, 2, 3, 4, 5, 6, 7], [2, 1, 2, 1, 2, 2, 1, 2,])

Map.addLayer(lulc_2023, {'min': 1, 'max': 2, 'palette': legend_colors}, 'LULC_2023')
forest_2023 = lulc_2023.updateMask(lulc_2023.neq(2))
Map.addLayer(forest_2023, {'palette': 'green'}, 'forest_2023')


In [ ]:
#calculate area of region and forest area and print it on the screen
stats = difference_image.multiply(ee.Image.pixelArea()).reduceRegion(
  reducer= ee.Reducer.sum(),
  geometry= aoi,
  scale= 30,
  maxPixels= 1e13,
  tileScale= 30);

difference_image = ee.Number(stats.get('sum')).divide(1000000).round();
print('Forest Area (sq.km.) in  2023', difference_image);

In [188]:
mask1=forest_2013.mask()
mask2=forest_2023.mask()
coverage_difference_mask = mask1.neq(mask2)
difference_image = forest_2013.updateMask(coverage_difference_mask)
#change_mask = lulc_2023.subtract(lulc_2013).gt(0).rename('change_detection')
#change_detection = forest_2013.updateMask(change_mask)
#change_detection = forest_2023.subtract(forest_2013).neq(0).rename('change_detection')
#change_detection = forest_2023.subtract(forest_2013).abs().rename('ChangeDetection')
#change_detection = forest_2023.lt(forest_2013)
#decreased_area_image2 = forest_2023.updateMask(change_detection)
#coverage_difference_mask = forest_2023.mask().eq(forest_2013.mask()).clip(aoi)
Map.addLayer(difference_image, {'palette': 'red'}, 'forest_change')
Map

Map(bottom=107634.0, center=[30.970544333076077, 78.69232152304562], controls=(WidgetControl(options=['positio…

In [23]:
geom=aoi.geometry()
out_dir = r'D:\Ankan\Extra\recruitment\amrut'
out_file = os.path.join(out_dir, 'landcover_2013.tif')
geemap.ee_export_image(forest_2013, filename=out_file, scale=90, region=geom)
out_file = os.path.join(out_dir, 'landcover_2023.tif')
geemap.ee_export_image(forest_2023, filename=out_file, scale=90, region=geom)

Generating URL ...
Please wait ...
Data downloaded to D:\Ankan\Extra\recruitment\amrut\landcover_2013.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Ankan\Extra\recruitment\amrut\landcover_2023.tif
